# Atalanta 2010-2020 from Serie B to Champions

This notebook uses data from the [https://www.football-data.co.uk]

In [1]:
import datetime
print(f"Last executed {datetime.datetime.now()}")

Last executed 2020-03-24 22:45:13.595916


In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

ModuleNotFoundError: No module named 'plotly'

## Importing data

In [ ]:
seasons = []
for year in range(10,19):
    seasons.append(str(year)+str(year+1))

In [ ]:
def geturl(var):
    return f"https://www.football-data.co.uk/mmz4281/{var}/I1.csv"

li = []
for season in seasons:
    df = pd.read_csv(geturl(season), index_col='Date', parse_dates=['Date']) 
                     #usecols=["Date", "HomeTeam", "AwayTeam", "FTHG"])     
    li.append(df)

df = pd.concat(li, sort=False)
df

In [ ]:
# Div = League Division
# Date = Match Date (dd/mm/yy)
# Time = Time of match kick off
# HomeTeam = Home Team
# AwayTeam = Away Team
# FTHG and HG = Full Time Home Team Goals
# FTAG and AG = Full Time Away Team Goals
# FTR and Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)
# HTHG = Half Time Home Team Goals
# HTAG = Half Time Away Team Goals
# HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

# Match Statistics (where available)
# Attendance = Crowd Attendance
# Referee = Match Referee
# HS = Home Team Shots
# AS = Away Team Shots
# HST = Home Team Shots on Target
# AST = Away Team Shots on Target
# HHW = Home Team Hit Woodwork
# AHW = Away Team Hit Woodwork
# HC = Home Team Corners
# AC = Away Team Corners
# HF = Home Team Fouls Committed
# AF = Away Team Fouls Committed
# HFKC = Home Team Free Kicks Conceded
# AFKC = Away Team Free Kicks Conceded
# HO = Home Team Offsides
# AO = Away Team Offsides
# HY = Home Team Yellow Cards
# AY = Away Team Yellow Cards
# HR = Home Team Red Cards
# AR = Away Team Red Cards
# HBP = Home Team Bookings Points (10 = yellow, 25 = red)
# ABP = Away Team Bookings Points (10 = yellow, 25 = red)

Choose below which data to consider.  The cell will select the appropriate row for the table, and only consider columns from the fifth onwards (i.e. ignoring the first four) 

In [ ]:
#FTHG and HG = Full Time Home Team Goals
#FTAG and AG = Full Time Away Team Goals
#FTR and Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)
team = "Atalanta"
home = df.loc[:,"HomeTeam"] == team
away = df.loc[:,"AwayTeam"] == team
cases = df[home|away].iloc[:,1:22]
dates = cases.index
cases

## Visualization